In [1]:
import os
import pickle
os.chdir('../')
from mtDNAsim import *
import pandas as pd
import numpy as np
from Bio import Phylo

/home/wangkun/mtDNA_simulation/mtDNAsim/sim_utils.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [18]:
from tqdm.notebook import tqdm

In [2]:
def DNAmutation(tree, mut_rate=0.1):
    mutations = dict()
    global_mutid = -1
    for i in tree.get_terminals():
        mut = []
        for j in tree.get_path(i):
            if j in mutations:
                mut = deepcopy(mutations[j])
            else:
                for _ in range(np.random.poisson(mut_rate*j.branch_length)):
                    mut.append(global_mutid+1)
                    global_mutid += 1
                mutations[j] = deepcopy(mut)

    mut_table = []
    cell_names = []
    for i in tree.get_terminals():
        seq = np.zeros(global_mutid+1)
        seq[mutations[i]]=1
        mut_table.append(seq)
        cell_names.append(i.name)
    return pd.DataFrame(np.array(mut_table), index=cell_names)

In [19]:
path = '/data3/wangkun/mtsim_res/20240903'
# for model in os.listdir(path):
#     if model == 'test':
#         continue
#     for simid in tqdm(os.listdir(f'{path}/{model}/')):
#         for gen in ['_330']:
#             tree = Phylo.read(f'{path}/{model}/{simid}/gt_tree{gen}.nwk', 'newick')
#             ndna_mut = DNAmutation(tree, mut_rate=0.8)
#             seqs = ndna_mut.astype(int)
#             with open(f'{path}/{model}/{simid}/dna_mut{gen}.phy', 'w') as f:
#                 f.write('{} {}\n'.format(*seqs.shape))
#                 for cell in seqs.index:
#                     f.write('{} {}\n'.format(cell, ''.join(seqs.loc[cell].astype(str)).replace('1', 'A').replace('0', 'G')))
#             brea

  0%|          | 0/20 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/data3/wangkun/mtsim_res/20240903/mid_0/93526/gt_tree_130.nwk'

In [31]:
from ete3 import Tree
from Bio import Phylo
from io import StringIO
import re
def robinson_foulds(tree1:'Bio.Phylo.BaseTree', tree2:'Bio.Phylo.BaseTree'):
    f = StringIO()
    Phylo.write(tree1, f, 'newick')
    tree1 = Tree(f.getvalue(), format=1)
    # tree1.prune(tree1.get_leaf_names())
    f = StringIO()
    Phylo.write(tree2, f, 'newick')
    tree2 = Tree(f.getvalue(), format=1)
    # tree2.prune(tree2.get_leaf_names())
    try:
        return tree1.robinson_foulds(tree2)
    except:
        return tree1.robinson_foulds(tree2, unrooted_trees=True)

In [48]:
simid

'152872'

In [49]:
path

'/data3/wangkun/mtsim_res/20240903'

In [53]:
with open(f'{path}/{model}/{simid}/rf_dist.txt', 'w') as f:
    f.write(f'model\ttype\tseq\tgen\tcutoff\trf\tmaxrf\n')
with open(f'{path}/{model}/{simid}/rf_dist.txt', 'a') as f:
    for gen in ['30', '130', '330']:
        if gen == '30':
            tree_gt = Phylo.read(f'{path}/{model}/{simid}/gt_tree.nwk', 'newick')
            for i in tree_gt.get_terminals():
                for j in tree_gt.get_path(i)[::-1][1:]:
                    if len(j.clades) == 1:
                        tree_gt.collapse(j)
            tree_ndna = Phylo.read(f'{path}/{model}/{simid}/dna_mut.phy_nj.nwk', 'newick')
        else:
            tree_gt = Phylo.read(f'{path}/{model}/{simid}/gt_tree_{gen}.nwk', 'newick')
            tree_ndna = Phylo.read(f'{path}/{model}/{simid}/dna_mut_{gen}.phy_nj.nwk', 'newick')
        rf_ndna = robinson_foulds(tree_gt, tree_ndna)
        f.write(f"{model}\tnDNA\t0\t{gen}\t0\t{rf_ndna[0]}\t{rf_ndna[1]}\n")
        for seq in ['', '_seq']:
            for cutoff in [0, 0.01]:
                tree_mt = Phylo.read(f'{path}/{model}/{simid}/mt_allmuts_{gen}_{cutoff}.phy_nj.nwk', 'newick')
                rf_mt = robinson_foulds(tree_gt, tree_mt)
                f.write(f"{model}\tmtDNA\t{seq}\t{gen}\t{cutoff}\t{rf_mt[0]}\t{rf_mt[1]}\n")
        

In [30]:
model = 'const_0'
simid = '152872'

In [32]:
gen = ''

In [35]:
tree_gt = Phylo.read(f'{path}/{model}/{simid}/gt_tree{gen}.nwk', 'newick')
for i in tree_gt.get_terminals():
    for j in tree_gt.get_path(i)[::-1][1:]:
        if len(j.clades) == 1:
            tree_gt.collapse(j)

In [37]:
tree_ndna = Phylo.read(f'{path}/{model}/{simid}/dna_mut.phy_nj.nwk', 'newick')

In [41]:
tree_mt0 = Phylo.read(f'{path}/{model}/{simid}/mt_allmuts_30_0.phy_nj.nwk', 'newick')

In [44]:
tree_mt1 = Phylo.read(f'{path}/{model}/{simid}/mt_allmuts_30_0.01.phy_nj.nwk', 'newick')

In [39]:
rf_ndna = robinson_foulds(tree_gt, tree_ndna)

In [42]:
rf_mt0 = robinson_foulds(tree_gt, tree_mt0)

In [46]:
rf_mt1 = robinson_foulds(tree_gt, tree_mt1)

In [40]:
rf_ndna[0]/rf_ndna[1]

0.14944834503510532

In [43]:
rf_mt0[0]/rf_mt0[1]

0.7342026078234705

In [47]:
rf_mt1[0]/rf_mt1[1]

0.921765295887663